# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [2]:
team_metro_dic = {'Tampa Bay Lightning':'Tampa Bay Area',
            'Boston Bruins':'Boston',
            'Toronto Maple Leafs':'Toronto',
            'Florida Panthers':'Miami–Fort Lauderdale',
            'Detroit Red Wings':'Detroit',
            'Montreal Canadiens':'Montreal',
            'Ottawa Senators':'Ottawa',
            'Buffalo Sabres':'Buffalo',
            'Washington Capitals':'Washington, D.C.',
            'Pittsburgh Penguins':'Pittsburgh',
            'Philadelphia Flyers':'Philadelphia',
            'Columbus Blue Jackets':'Columbus',
            'New Jersey Devils':'New York City',
            'Carolina Hurricanes':'Raleigh',
            'New York Islanders':'New York City',
            'New York Rangers':'New York City',
            'Nashville Predators':'Nashville',
            'Winnipeg Jets':'Winnipeg',
            'Minnesota Wild':'Minneapolis–Saint Paul',
            'Colorado Avalanche':'Denver',
            'St. Louis Blues':'St. Louis',
            'Dallas Stars':'Dallas–Fort Worth',
            'Chicago Blackhawks':'Chicago',
            'Vegas Golden Knights':'Las Vegas',
            'Anaheim Ducks':'Los Angeles',
            'San Jose Sharks':'San Francisco Bay Area',
            'Los Angeles Kings':'Los Angeles',
            'Calgary Flames':'Calgary',
            'Edmonton Oilers':'Edmonton',
            'Vancouver Canucks':'Vancouver',
            'Arizona Coyotes':'Phoenix',
            'Arizona Diamondbacks':'Phoenix',
            'Atlanta Braves':'Atlanta',
            'Baltimore Orioles':'Baltimore',
            'Boston Red Sox':'Boston',
            'Chicago Cubs':'Chicago',
            'Chicago White Sox':'Chicago',
            'Cincinnati Reds':'Cincinnati',
            'Cleveland Indians':'Cleveland',
            'Colorado Rockies':'Denver',
            'Detroit Tigers':'Detroit',
            'Houston Astros':'Houston',
            'Kansas City Royals':'Kansas City',
            'Los Angeles Angels':'Los Angeles',
            'Los Angeles Dodgers':'Los Angeles',
            'Miami Marlins':'Miami–Fort Lauderdale',
            'Milwaukee Brewers':'Milwaukee',
            'Minnesota Twins':'Minneapolis–Saint Paul',
            'New York Mets':'New York City',
            'New York Yankees':'New York City',
            'Oakland Athletics':'San Francisco Bay Area',
            'Philadelphia Phillies':'Philadelphia',
            'Pittsburgh Pirates':'Pittsburgh',
            'San Diego Padres':'San Diego',
            'San Francisco Giants':'San Francisco Bay Area',
            'Seattle Mariners':'Seattle',
            'St. Louis Cardinals':'St. Louis',
            'Tampa Bay Rays':'Tampa Bay Area',
            'Texas Rangers':'Dallas–Fort Worth',
            'Toronto Blue Jays':'Toronto',
            'Washington Nationals':'Washington, D.C.',
            'Arizona Cardinals':'Phoenix',
            'Atlanta Falcons':'Atlanta',
            'Baltimore Ravens':'Baltimore',
            'Buffalo Bills':'Buffalo',
            'Carolina Panthers':'Charlotte',
            'Chicago Bears':'Chicago',
            'Cincinnati Bengals':'Cincinnati',
            'Cleveland Browns':'Cleveland',
            'Dallas Cowboys':'Dallas–Fort Worth',
            'Denver Broncos':'Denver',
            'Detroit Lions':'Detroit',
            'Green Bay Packers':'Green Bay',
            'Houston Texans':'Houston',
            'Indianapolis Colts':'Indianapolis',
            'Jacksonville Jaguars':'Jacksonville',
            'Kansas City Chiefs':'Kansas City',
            'Los Angeles Chargers':'Los Angeles',
            'Los Angeles Rams':'Los Angeles',
            'Miami Dolphins':'Miami–Fort Lauderdale',
            'Minnesota Vikings':'Minneapolis–Saint Paul',
            'New England Patriots':'Boston',
            'New Orleans Saints':'New Orleans',
            'New York Giants':'New York City',
            'New York Jets':'New York City',
            'Oakland Raiders':'San Francisco Bay Area',
            'Philadelphia Eagles':'Philadelphia',
            'Pittsburgh Steelers':'Pittsburgh',
            'San Francisco 49ers':'San Francisco Bay Area',
            'Seattle Seahawks':'Seattle',
            'Tampa Bay Buccaneers':'Tampa Bay Area',
            'Tennessee Titans':'Nashville',
            'Washington Redskins':'Washington, D.C.',
            'Toronto Raptors':'Toronto',
            'Boston Celtics':'Boston',
            'Philadelphia 76ers':'Philadelphia',
            'Cleveland Cavaliers':'Cleveland',
            'Indiana Pacers':'Indianapolis',
            'Miami Heat':'Miami–Fort Lauderdale',
            'Milwaukee Bucks':'Milwaukee',
            'Washington Wizards':'Washington, D.C.',
            'Detroit Pistons':'Detroit',
            'Charlotte Hornets':'Charlotte',
            'New York Knicks':'New York City',
            'Brooklyn Nets':'New York City',
            'Chicago Bulls':'Chicago',
            'Orlando Magic':'Orlando',
            'Atlanta Hawks':'Atlanta',
            'Houston Rockets':'Houston',
            'Golden State Warriors':'San Francisco Bay Area',
            'Portland Trail Blazers':'Portland',
            'Oklahoma City Thunder':'Oklahoma City',
            'Utah Jazz':'Salt Lake City',
            'New Orleans Pelicans':'New Orleans',
            'San Antonio Spurs':'San Antonio',
            'Minnesota Timberwolves':'Minneapolis–Saint Paul',
            'Denver Nuggets':'Denver',
            'Los Angeles Clippers':'Los Angeles',
            'Los Angeles Lakers':'Los Angeles',
            'Sacramento Kings':'Sacramento',
            'Dallas Mavericks':'Dallas–Fort Worth',
            'Memphis Grizzlies':'Memphis',
            'Phoenix Suns':'Phoenix'}

In [3]:
team_metro_df = pd.DataFrame(data=team_metro_dic, index=[0]).T
team_metro_df = team_metro_df.reset_index().rename(columns={'index':'team',0:'Metropolitan area'})

In [4]:
mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [5]:
city_popul = cities.copy()
city_popul.replace('\[[\w\s]*\]','',regex=True, inplace=True)
city_popul.replace('\s*\—\s*','',regex=True, inplace=True)
city_popul.replace('',np.nan, inplace= True)
city_popul.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace=True)
city_popul = city_popul.loc[:,['Metropolitan area','Population']]

In [6]:
#NHL pre-processing
nhl_df_2018 = nhl_df.copy()
nhl_df_2018 = nhl_df_2018.loc[:,['team','W','L','year','League']]
nhl_df_2018 = nhl_df_2018[nhl_df_2018['year']==2018]
nhl_df_2018.replace('[A-Za-z]*\sDivision',np.nan,regex=True, inplace=True)
nhl_df_2018.replace('\*','',regex=True, inplace=True)
nhl_df_2018 = nhl_df_2018.dropna()
nhl_df_2018 = nhl_df_2018.merge(team_metro_df, on='team', how='left').merge(city_popul, on='Metropolitan area', how='left')

nhl_df_2018['W'] = pd.to_numeric(nhl_df_2018['W'])
nhl_df_2018['L'] = pd.to_numeric(nhl_df_2018['L'])
nhl_df_2018['Population'] = pd.to_numeric(nhl_df_2018['Population'])

nhl_df_2018['win/total'] = nhl_df_2018['W']/(nhl_df_2018['W']+nhl_df_2018['L'])

In [7]:
#NBA pre-processing
nba_df_2018 = nba_df.copy()
nba_df_2018 = nba_df_2018.loc[:,['team','W','L','year','League']]
nba_df_2018 = nba_df_2018[nba_df_2018['year']==2018]
nba_df_2018.replace('[\W]*\([0-9]*\)','',regex=True, inplace=True)
nba_df_2018 = nba_df_2018.merge(team_metro_df, on='team', how='left').merge(city_popul, on='Metropolitan area', how='left')

nba_df_2018['W'] = pd.to_numeric(nba_df_2018['W'])
nba_df_2018['L'] = pd.to_numeric(nba_df_2018['L'])
nba_df_2018['Population'] = pd.to_numeric(nba_df_2018['Population'])

nba_df_2018['win/total'] = nba_df_2018['W']/(nba_df_2018['W']+nba_df_2018['L'])

In [8]:
#MLB pre-processing
mlb_df_2018 = mlb_df.copy()
mlb_df_2018 = mlb_df_2018.loc[:,['team','W','L','year','League']]
mlb_df_2018 = mlb_df_2018[mlb_df_2018['year']==2018]
mlb_df_2018 = mlb_df_2018.merge(team_metro_df, on='team', how='left').merge(city_popul, on='Metropolitan area', how='left')

mlb_df_2018["W"] = pd.to_numeric(mlb_df_2018['W'])
mlb_df_2018["L"] = pd.to_numeric(mlb_df_2018['L'])
mlb_df_2018["Population"] = pd.to_numeric(mlb_df_2018['Population'])

mlb_df_2018['win/total'] = mlb_df_2018['W']/(mlb_df_2018['W']+mlb_df_2018['L'])

In [9]:
#NFL pre-processing
nfl_df_2018 = nfl_df.copy()
nfl_df_2018 = nfl_df_2018.loc[:,['team','W','L','year','League']]
nfl_df_2018 = nfl_df_2018[nfl_df_2018['year']==2018]
nfl_df_2018.replace('(AFC|NFC)\s(East|North|South|West)','',regex=True, inplace=True)
nfl_df_2018.replace('\*|\+', '', regex=True, inplace=True)
nfl_df_2018.replace('',np.nan, inplace=True)
nfl_df_2018 = nfl_df_2018.dropna()
nfl_df_2018 = nfl_df_2018.merge(team_metro_df, on='team', how='left').merge(city_popul, on='Metropolitan area', how='left')

nfl_df_2018['W'] = pd.to_numeric(nfl_df_2018['W'])
nfl_df_2018['L'] = pd.to_numeric(nfl_df_2018['L'])
nfl_df_2018['Population'] = pd.to_numeric(nfl_df_2018['Population'])

nfl_df_2018['win/total'] = nfl_df_2018['W']/(nfl_df_2018['W']+nfl_df_2018['L'])

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [10]:
def nhl_correlation(): 
    # YOUR CODE HERE
    nhl_df_final = nhl_df_2018.groupby('Metropolitan area').mean()
    
    
    population_by_region = nhl_df_final['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nhl_df_final['win/total'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [11]:
nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [12]:
def nba_correlation():
    # YOUR CODE HERE
    nba_df_final = nba_df_2018.groupby('Metropolitan area').mean()
    
    population_by_region =  nba_df_final['Population'] # pass in metropolitan area population from cities
    win_loss_by_region =  nba_df_final['win/total'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [13]:
nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [14]:
def mlb_correlation(): 
    # YOUR CODE HERE
    mlb_df_final = mlb_df_2018.groupby('Metropolitan area').mean()
    
    population_by_region = mlb_df_final['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = mlb_df_final['win/total'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [15]:
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [16]:
def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_df_final = nfl_df_2018.groupby('Metropolitan area').mean()
    
    population_by_region = nfl_df_final['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nfl_df_final['win/total'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [17]:
nfl_correlation()

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [50]:
def sports_team_performance():
    # YOUR CODE HERE
    #NHL
    nhl_df_final = nhl_df_2018.copy()

    #NBA
    nba_df_final = nba_df_2018.copy()

    #MLB
    mlb_df_final = mlb_df_2018.copy()

    #NFL
    nfl_df_final = nfl_df_2018.copy()
    
    All_sports_2018 = pd.concat([nhl_df_final, nba_df_final, mlb_df_final, nfl_df_final],
                                axis=0, join = 'outer',ignore_index = True)
    
    All_sports_2018 = All_sports_2018.set_index(['League','Metropolitan area']).sort_index(level='League').drop(columns='Population')
    
    All_sports_final = All_sports_2018.groupby(['League','Metropolitan area']).mean()
    
    All_mlb = All_sports_final.loc['MLB']
    All_nfl = All_sports_final.loc['NFL']
    All_nhl = All_sports_final.loc['NHL']
    All_nba = All_sports_final.loc['NBA']
    
    NFL_NHL = All_nfl.merge(All_nhl, how='inner',on='Metropolitan area', suffixes=['_nfl','_nhl'])
    NFL_NBA = All_nfl.merge(All_nba, how='inner',on='Metropolitan area', suffixes=['_nfl','_nba'])
    MLB_NFL = All_mlb.merge(All_nfl, how='inner',on='Metropolitan area', suffixes=['_mlb','_nfl'])
    MLB_NHL = All_mlb.merge(All_nhl, how='inner',on='Metropolitan area', suffixes=['_mlb','_nhl'])
    MLB_NBA = All_mlb.merge(All_nba, how='inner',on='Metropolitan area', suffixes=['_mlb','_nba'])
    NHL_NBA = All_nhl.merge(All_nba, how='inner',on='Metropolitan area', suffixes=['_nhl','_nba'])
    
    NFL_NHL_value = stats.ttest_rel(NFL_NHL['win/total_nfl'], NFL_NHL['win/total_nhl'])
    NFL_NBA_value = stats.ttest_rel(NFL_NBA['win/total_nfl'], NFL_NBA['win/total_nba'])
    MLB_NFL_value = stats.ttest_rel(MLB_NFL['win/total_mlb'], MLB_NFL['win/total_nfl'])
    MLB_NHL_value = stats.ttest_rel(MLB_NHL['win/total_mlb'], MLB_NHL['win/total_nhl'])
    MLB_NBA_value = stats.ttest_rel(MLB_NBA['win/total_mlb'], MLB_NBA['win/total_nba'])
    NHL_NBA_value = stats.ttest_rel(NHL_NBA['win/total_nhl'], NHL_NBA['win/total_nba'])
    
    NFL_NHL_result = NFL_NHL_value[1]
    NFL_NBA_result = NFL_NBA_value[1]
    MLB_NFL_result = MLB_NFL_value[1]
    MLB_NHL_result = MLB_NHL_value[1]
    MLB_NBA_result = MLB_NBA_value[1]
    NHL_NBA_result = NHL_NBA_value[1]
       
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame(data=[[np.nan,NFL_NBA_result,NFL_NHL_result, MLB_NFL_result],
                            [NFL_NBA_result, np.nan, NHL_NBA_result, MLB_NBA_result],
                            [NFL_NHL_result, NHL_NBA_result, np.nan, MLB_NHL_result],
                            [MLB_NFL_result, MLB_NBA_result, MLB_NHL_result, np.nan]],
                         index = sports, columns = sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [51]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
